In [ ]:
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow.keras.backend as K
from tensorflow import keras
from sklearn import model_selection
import imgaug.augmenters as iaa
import imgaug as ia
import os

In [ ]:
ia.seed(1)
def Data_Aug(samples):
    randomaug = iaa.Sometimes(
        0.5,
        iaa.OneOf([
            iaa.Sequential([
                iaa.AllChannelsCLAHE(clip_limit=(1, 10)),
                iaa.Affine(rotate=(-5, 5))
            ]),
            iaa.Sequential([
                iaa.Affine(translate_percent={
                    "x": (-0.2, 0.2),
                    "y": (-0.2, 0.2)
                }),
                iaa.Multiply((0.7, 1.3)),
                iaa.AdditiveLaplaceNoise(scale=0.03 * 255, per_channel=True),
                iaa.Fliplr(0.3),
                iaa.Flipud(0.3)
            ]),
            iaa.Sequential([
                iaa.Affine(rotate=(-10, 10)),
                iaa.Multiply((0.7, 1.3)),
                iaa.AdditiveLaplaceNoise(scale=0.03 * 255, per_channel=True),
                iaa.Fliplr(0.3),
                iaa.Flipud(0.3)
            ]),
            iaa.Sequential([
                iaa.PerspectiveTransform(scale=(0, 0.15)),
                iaa.Multiply((0.7, 1.3)),
                iaa.AdditiveLaplaceNoise(scale=0.03 * 255, per_channel=True),
                iaa.Fliplr(0.3),
                iaa.Flipud(0.3)
            ])
        ]))


    return randomaug.augment_images(samples)

def resize_images(imgs, hight, width):
    seq = iaa.Resize({"height": hight, "width": width})
    aug = seq.augment_images(imgs)
    return aug

In [ ]:
class cifar10data(keras.utils.Sequence):
    '''
    generate data batch of cifar10
    10 classes
    '''
    def __init__(self, data_x, data_y, batch_size, train):
        self.data_x = data_x
        self.data_y = data_y
        self.train = train
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.data_y) / self.batch_size))

    def __getitem__(self, idx):

        batch_x = self.data_x[idx * self.batch_size:(idx + 1) *
                              self.batch_size]
        batch_y = self.data_y[idx * self.batch_size:(idx + 1) *
                              self.batch_size]
        if self.train == True:
            batch_x = Data_Aug(batch_x)

        batch_x = np.clip(batch_x / 255.0, 0, 1).astype("float32")
        batch_y = keras.utils.to_categorical(np.array(batch_y), 10)
        return batch_x, batch_y

In [ ]:
class CustomEarlyStoppingAndSave(keras.callbacks.Callback):
    def __init__(self, save_path, patience=0):
        super(CustomEarlyStoppingAndSave, self).__init__()
        #self.patience = patience
        self.save_path = save_path
        self.best_weights = None

    def on_train_begin(self, logs=None):
        #self.wait = 0
        self.stopped_epoch = 0
        #self.best_val_distance_loss = np.Inf
        self.best_val_acc = 0

    def on_epoch_end(self, epoch, logs=None):
        #val_distance_loss = logs.get('val_MMD_loss')
        val_acc = logs.get('val_accuracy') 

        if  np.greater(val_acc, self.best_val_acc):
            #self.best_val_distance_loss = val_distance_loss
            self.best_val_acc = val_acc
            #self.wait = 0
            # Record the best weights if current results is better (less).
            self.best_weights = self.model.get_weights()
            self.stopped_epoch = epoch
        #else:
            #self.wait += 1
            #if self.wait >= self.patience:
                #self.stopped_epoch = epoch
                #self.model.stop_training = True
                #print("Restoring model weights from the end of the best epoch.")
                #self.model.set_weights(self.best_weights)

    def on_train_end(self, logs=None):
        self.model.set_weights(self.best_weights)
        if self.stopped_epoch >= 0:
            print("Epoch %05d: early stopping and save the model" %
                  (self.stopped_epoch + 1))
        self.model.save(self.save_path)

In [ ]:
def create_VGGNet(input_shape, num_classes, random_seed):
  inputs = keras.Input(shape=input_shape)
  x = keras.layers.Conv2D(64, (3, 3), padding='same',  kernel_initializer=keras.initializers.HeUniform(seed=random_seed))(inputs)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.Activation('relu')(x)
  x = keras.layers.Conv2D(64, (3, 3), padding='same',  kernel_initializer=keras.initializers.HeUniform(seed=random_seed))(x)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.Activation('relu')(x)
  p1 = keras.layers.MaxPooling2D((2, 2))(x)
  x = keras.layers.Dropout(0.2)(p1)

  x = keras.layers.Conv2D(128, (3, 3), padding='same',  kernel_initializer=keras.initializers.HeUniform(seed=random_seed))(x)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.Activation('relu')(x)
  x = keras.layers.Conv2D(128, (3, 3), padding='same',  kernel_initializer=keras.initializers.HeUniform(seed=random_seed))(x)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.Activation('relu')(x)
  p2 = keras.layers.MaxPooling2D((2, 2))(x)
  x = keras.layers.Dropout(0.3)(p2)

  '''  
  x = keras.layers.Conv2D(256, (3, 3), padding='same',  kernel_initializer=keras.initializers.HeUniform(seed=random_seed))(x)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.Activation('relu')(x)
  f6 = keras.layers.Conv2D(256, (3, 3), padding='same',  kernel_initializer=keras.initializers.HeUniform(seed=random_seed))(x)
  x = keras.layers.BatchNormalization()(f6)
  r3 = keras.layers.Activation('relu')(x)
  p3 = keras.layers.MaxPooling2D((2, 2))(r3)
  x = keras.layers.Dropout(0.4)(p3)
  '''

  x = keras.layers.Flatten()(x)
  x = keras.layers.Dense(128,  kernel_initializer=keras.initializers.HeUniform(seed=random_seed))(x)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.Activation('relu')(x)
  x = keras.layers.Dropout(0.5)(x)
  outputs = keras.layers.Dense(num_classes, activation='softmax', kernel_initializer=keras.initializers.HeUniform(seed=random_seed))(x)
  return keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
#create single base ResNet architecture
def creat_ResNet(input_shape, num_classes=10, num_filters = 64, random_seed=None):
  inputs = keras.Input(shape=input_shape)
  y = keras.layers.Conv2D(num_filters, (3, 3), padding='same', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(inputs)
  y = keras.layers.BatchNormalization()(y)
  y = keras.layers.LeakyReLU(alpha=0.1)(y)
  y = keras.layers.Conv2D(num_filters, (3, 3), padding='same', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(y)
  y = keras.layers.BatchNormalization()(y)
  y = keras.layers.LeakyReLU(alpha=0.1)(y)
  y = keras.layers.Conv2D(num_filters, (3, 3), padding='same', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(y)
  y = keras.layers.BatchNormalization()(y)
  y = keras.layers.LeakyReLU(alpha=0.1)(y)
  x = keras.layers.Conv2D(num_filters, (1, 1), kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(inputs)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.add([x, y])
  x = keras.layers.LeakyReLU(alpha=0.1)(x)
  x = keras.layers.MaxPooling2D((2,2))(x)

  y = keras.layers.Conv2D(2*num_filters, (3, 3), padding='same', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(x)
  y = keras.layers.BatchNormalization()(y)
  y = keras.layers.LeakyReLU(alpha=0.1)(y)
  y = keras.layers.Conv2D(2*num_filters, (3, 3), padding='same', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(y)
  y = keras.layers.BatchNormalization()(y)
  y = keras.layers.LeakyReLU(alpha=0.1)(y)
  y = keras.layers.Conv2D(2*num_filters, (3, 3), padding='same', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(y)
  y = keras.layers.BatchNormalization()(y)
  y = keras.layers.LeakyReLU(alpha=0.1)(y)
  x = keras.layers.Conv2D(2*num_filters, (1, 1), kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(x)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.add([x, y])
  x = keras.layers.LeakyReLU(alpha=0.1)(x)
  x = keras.layers.MaxPooling2D((2,2))(x) 

  y = keras.layers.Conv2D(4*num_filters, (3, 3), padding='same', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(x)
  y = keras.layers.BatchNormalization()(y)
  y = keras.layers.LeakyReLU(alpha=0.1)(y)
  y = keras.layers.Conv2D(4*num_filters, (3, 3), padding='same', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(y)
  y = keras.layers.BatchNormalization()(y)
  y = keras.layers.LeakyReLU(alpha=0.1)(y)
  y = keras.layers.Conv2D(4*num_filters, (3, 3), padding='same', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(y)
  y = keras.layers.BatchNormalization()(y)
  y = keras.layers.LeakyReLU(alpha=0.1)(y)
  x = keras.layers.Conv2D(4*num_filters, (1, 1), kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(x)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.add([x, y])
  x = keras.layers.LeakyReLU(alpha=0.1)(x)
  last_acti = keras.layers.MaxPooling2D((2,2))(x) 

  '''  
  y = keras.layers.Conv2D(8*num_filters, (3, 3), padding='same', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(x)
  y = keras.layers.BatchNormalization()(y)
  y = keras.layers.LeakyReLU(alpha=0.1)(y)
  y = keras.layers.Conv2D(8*num_filters, (3, 3), padding='same', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(y)
  y = keras.layers.BatchNormalization()(y)
  y = keras.layers.LeakyReLU(alpha=0.1)(y)
  y = keras.layers.Conv2D(8*num_filters, (3, 3), padding='same', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(y)
  y = keras.layers.BatchNormalization()(y)
  y = keras.layers.LeakyReLU(alpha=0.1)(y)
  x = keras.layers.Conv2D(8*num_filters, (1, 1), kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(x)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.add([x, y])
  x = keras.layers.LeakyReLU(alpha=0.1)(x)
  last_acti = keras.layers.MaxPooling2D((2,2))(x) 
  '''

  x = keras.layers.GlobalAveragePooling2D()(last_acti)
  outputs = keras.layers.Dense(num_classes, activation='softmax', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(x)
  model = keras.Model(inputs=inputs, outputs=[outputs,last_acti])
  return model


In [ ]:
def create_AlexNet(input_shape, num_classes=10, random_seed=None):
  inputs = keras.Input(shape=input_shape)
  x = keras.layers.Conv2D(96, (3,3), padding='same', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(inputs)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.Activation('relu')(x)
  x = keras.layers.MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='same')(x)
  
  x = keras.layers.Conv2D(256, (3,3), padding='same', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(x)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.Activation('relu')(x) 
  x = keras.layers.MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='same')(x)
  
  x = keras.layers.Conv2D(384, (3,3), padding='same', activation='relu', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(x) 
  x = keras.layers.Conv2D(384, (3,3), padding='same', activation='relu', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(x) 
  x = keras.layers.Conv2D(256, (3,3), padding='same', activation='relu', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(x)
  feature_maps = keras.layers.MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='same')(x)

  x = keras.layers.Flatten()(feature_maps)
  #x = keras.layers.Dense(256,activation='relu')(x)
  #x = keras.layers.Dropout(0.5)(x)
  x = keras.layers.Dense(128,activation='relu')(x)
  x = keras.layers.Dropout(0.5)(x)
  outputs = keras.layers.Dense(num_classes, activation='softmax')(x)
  model = keras.Model(inputs=inputs, outputs=outputs)
  return model

In [ ]:
input_size= (32,32,3)
model = create_VGGNet(input_shape=input_size, num_classes=10,  random_seed=None)
model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3),
                    loss=['categorical_crossentropy'],
                    metrics=['accuracy'])

(x_origin_train, y_origin_train), (x_test,y_test) = keras.datasets.cifar10.load_data()



y_origin_train = y_origin_train.reshape(-1,)
y_test = y_test.reshape(-1,)

x_train, x_val, y_train, y_val = model_selection.train_test_split(
    x_origin_train,
    y_origin_train,
    test_size=0.25,
    random_state=0,
    stratify=y_origin_train)
_, x_val, _, y_val = model_selection.train_test_split(
    x_val,
    y_val,
    test_size=2000,
    random_state=0,
    stratify=y_val)

#randomly select training samples
train_index = []
random.seed(0)
for i in range(10):
    train_index += (random.sample(list(np.where(y_train == i)[0]), 100))
random.shuffle(train_index)
sampled_x_train = x_train[train_index]
sampled_y_train = y_train[train_index]
del train_index

train_sequence = cifar10data(sampled_x_train, sampled_y_train, batch_size=10, train=True)
validation_sequence = cifar10data(x_val, y_val, batch_size=10, train=False)

#train base models and save parameters
history = model.fit(
    train_sequence,
    epochs=300,
    validation_data=validation_sequence,
    #validation_steps=200,
    callbacks=CustomEarlyStoppingAndSave(
        save_path='3_samples_seed_fix.h5',
        patience=10))

170508288/170498071 [==============================] - 2s 0us/step
Epoch 1/300
100/100 [==============================] - 14s 27ms/step - loss: 2.5689 - accuracy: 0.1780 - val_loss: 2.5170 - val_accuracy: 0.1755
Epoch 2/300
100/100 [==============================] - 2s 24ms/step - loss: 2.1952 - accuracy: 0.2440 - val_loss: 1.9420 - val_accuracy: 0.3145
Epoch 3/300
100/100 [==============================] - 2s 24ms/step - loss: 2.0715 - accuracy: 0.2890 - val_loss: 1.8992 - val_accuracy: 0.3060
Epoch 4/300
100/100 [==============================] - 2s 23ms/step - loss: 2.0083 - accuracy: 0.2990 - val_loss: 2.0449 - val_accuracy: 0.2575
Epoch 5/300
100/100 [==============================] - 2s 24ms/step - loss: 1.8741 - accuracy: 0.3290 - val_loss: 1.7922 - val_accuracy: 0.3470
Epoch 6/300
100/100 [==============================] - 2s 24ms/step - loss: 1.7402 - accuracy: 0.3750 - val_loss: 1.7231 - val_accuracy: 0.3860
Epoch 7/300
100/100 [==============================] - 2s 25ms/step 

In [ ]:
x_test = x_test/255.0

In [ ]:
evaluate1 = model.evaluate(x_test,keras.utils.to_categorical(y_test),batch_size=10)
evaluate1

1000/1000 [==============================] - 3s 3ms/step - loss: 1.6682 - accuracy: 0.5559


[1.6681857109069824, 0.555899977684021]

In [ ]:
model_train = keras.models.load_model('3_samples_seed_fix.h5')
evaluate2 = model_train.evaluate(x_test,keras.utils.to_categorical(y_test),batch_size=10)
evaluate2

1000/1000 [==============================] - 3s 2ms/step - loss: 1.6682 - accuracy: 0.5559


[1.6681857109069824, 0.555899977684021]